# Collateral Value

In [23]:
from mantle import mantle
from env import contract_address
import numpy as np

user_wallet_address = 'terra12hnhh5vtyg5juqnzm43970nh4fw42pt27nw9g9'

result = mantle(
    wasm_query={
    'overseer_collaterals': {
      'contractAddress': contract_address['moneyMarket']['overseer'],
      'query': {
        'collaterals': {
          'borrower': user_wallet_address
        }
      }
    },
    'oracle_prices': {
      'contractAddress': contract_address['moneyMarket']['oracle'],
      'query': {
        'prices': {}
      }
    }
  }
)

print(result)

{'oracle_prices': {'Ok': {'prices': [{'asset': 'terra19mkj9nec6e3y5754tlnuz4vem7lzh4n0lc2s3l', 'last_updated_time': 1627812593, 'price': '2605.69271193'}, {'asset': 'terra1u0t35drzyy0mujj8rkdyzhe264uls4ug3wdp3x', 'last_updated_time': 1627812598, 'price': '11.050451749127510659'}]}}, 'overseer_collaterals': {'borrower': 'terra12hnhh5vtyg5juqnzm43970nh4fw42pt27nw9g9', 'collaterals': [['terra19mkj9nec6e3y5754tlnuz4vem7lzh4n0lc2s3l', '11350'], ['terra1u0t35drzyy0mujj8rkdyzhe264uls4ug3wdp3x', '10000000']]}}


In [35]:
if result['oracle_prices']['Ok'] is not None:
  oracle_prices = result['oracle_prices']['Ok']['prices']
else:
  oracle_prices = result['oracle_prices']['prices']

collateral_address = list(map(lambda oracle: oracle['asset'], oracle_prices))
collaterals = result['overseer_collaterals']['collaterals']

locked_amounts = []

for address in collateral_address:
  collateral = next((item for item in collaterals if item[0] == address), None)
  if collateral is not None:
    locked_amounts.append(float(collateral[1]))
  else:
    locked_amounts.append(0.0)

prices = np.array(list(map(lambda oracle: float(oracle['price']), oracle_prices)))
locked_amounts = np.array(locked_amounts)

print('collateral_address', collateral_address)

print('locked_amounts', locked_amounts)
print('prices', prices)

locked_amounts_in_ust = locked_amounts * prices
collateral_value = np.sum(locked_amounts_in_ust)

print('locked_amounts (in UST)', locked_amounts_in_ust / 1000000.0)
print('collateral_value (in UST)', collateral_value / 1000000.0)

collateral_address ['terra19mkj9nec6e3y5754tlnuz4vem7lzh4n0lc2s3l', 'terra1u0t35drzyy0mujj8rkdyzhe264uls4ug3wdp3x']
locked_amounts [   11350. 10000000.]
prices [2605.69271193   11.05045175]
locked_amounts (in UST) [ 29.57461228 110.50451749]
collateral_value (in UST) 140.0791297716806
